In [9]:
import cv2
import os
import numpy as np

In [10]:
def get_center(img):
    return (img.shape[1]//2, img.shape[0]//2)

def rotate_img(img, angle, scale=1):
    rotation_matrix = cv2.getRotationMatrix2D(get_center(img), angle, scale)
    rotated_img = cv2.warpAffine(img, rotation_matrix, (img.shape[1], img.shape[0]))
    return rotated_img

In [11]:
def composite_images_with_shadow(background_path, foreground_path, output_path, angle, shadow_offset=(5, 5), shadow_blur_kernel=(25, 25)):
    """
    배경 위에 그림자 효과와 함께 전경 이미지를 합성합니다.
    
    Args:
        background_path (str): 배경 이미지 파일 경로.
        foreground_path (str): 전경 이미지 파일 경로 (알파 채널 포함).
        output_path (str): 결과 이미지를 저장할 경로.
        shadow_offset (tuple): 그림자의 (x, y) 오프셋.
        shadow_blur_kernel (tuple): 그림자 블러 처리를 위한 가우시안 커널 크기.
    """
    background = cv2.imread(background_path)
    if background is None:
        print(f"오류: 배경 이미지 '{background_path}'를 로드할 수 없습니다.")
        return

    foreground = cv2.imread(foreground_path, cv2.IMREAD_UNCHANGED)
    if foreground is None:
        print(f"오류: 전경 이미지 '{foreground_path}'를 로드할 수 없습니다.")
        return

    foreground = rotate_img(foreground, angle)
    
    if foreground.shape[2] == 4:
        alpha_mask = foreground[:, :, 3]
        
        shadow = np.zeros_like(foreground)
        shadow[:, :, 3] = alpha_mask
        
        blurred_shadow = cv2.GaussianBlur(shadow, shadow_blur_kernel, 0)
        
        fg_height, fg_width, _ = foreground.shape
        bg_height, bg_width, _ = background.shape
        
        center_x = (bg_width - fg_width) // 2
        center_y = (bg_height - fg_height) // 2
        
        shadow_x = center_x + shadow_offset[0]
        shadow_y = center_y + shadow_offset[1]

        y1_s, y2_s = max(0, shadow_y), min(bg_height, shadow_y + fg_height)
        x1_s, x2_s = max(0, shadow_x), min(bg_width, shadow_x + fg_width)

        shadow_roi_bg = background[y1_s:y2_s, x1_s:x2_s]
        shadow_to_blend = blurred_shadow[max(0, -shadow_y):min(fg_height, bg_height - shadow_y), 
                                       max(0, -shadow_x):min(fg_width, bg_width - shadow_x)]

        shadow_alpha = (shadow_to_blend[:, :, 3] / 255.0).astype(np.float32)
        shadow_alpha_factor = cv2.cvtColor(shadow_alpha, cv2.COLOR_GRAY2BGR)
        shadow_rgb = shadow_to_blend[:, :, :3]
        
        composed_shadow_roi = shadow_rgb * shadow_alpha_factor + shadow_roi_bg * (1 - shadow_alpha_factor)
        background[y1_s:y2_s, x1_s:x2_s] = composed_shadow_roi.astype(np.uint8)

    if foreground.shape[2] == 4:
        fg_x = center_x
        fg_y = center_y

        y1_f, y2_f = max(0, fg_y), min(bg_height, fg_y + fg_height)
        x1_f, x2_f = max(0, fg_x), min(bg_width, fg_x + fg_width)
        
        fg_roi_bg = background[y1_f:y2_f, x1_f:x2_f]
        fg_to_blend = foreground[max(0, -fg_y):min(fg_height, bg_height - fg_y), 
                                 max(0, -fg_x):min(fg_width, bg_width - fg_x)]
        
        fg_alpha = (fg_to_blend[:, :, 3] / 255.0).astype(np.float32)
        fg_alpha_factor = cv2.cvtColor(fg_alpha, cv2.COLOR_GRAY2BGR)
        fg_rgb = fg_to_blend[:, :, :3]
        
        composed_fg_roi = fg_rgb * fg_alpha_factor + fg_roi_bg * (1 - fg_alpha_factor)
        background[y1_f:y2_f, x1_f:x2_f] = composed_fg_roi.astype(np.uint8)
    else:
        center_x = (bg_width - fg_width) // 2
        center_y = (bg_height - fg_height) // 2
        y1, y2 = max(0, center_y), min(bg_height, center_y + fg_height)
        x1, x2 = max(0, center_x), min(bg_width, center_x + fg_width)
        background[y1:y2, x1:x2] = foreground

    cv2.imwrite(output_path, background)
    print(f"그림자가 적용된 이미지가 '{output_path}'에 저장되었습니다.")


In [13]:
image_folder = "image/pressure_gauge"
background_image_path = os.path.join(image_folder, "pressure_gauge.png")
foreground_image_path = os.path.join(image_folder, "niddle.png")

for degree in range(0, -280, -5):
    output_image_path = f"composite/composited_gauge_with_shadow_{str(abs(degree))}.png"
    composite_images_with_shadow(background_image_path, foreground_image_path, output_image_path, angle=degree)

그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_0.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_5.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_10.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_15.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_20.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_25.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_30.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_35.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_40.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_45.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_50.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_55.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_60.png'에 저장되었습니다.
그림자가 적용된 이미지가 'composite/composited_gauge_with_shadow_65.png'에 저장되